# Texter with vanilla cnn

In [2]:
%matplotlib inline
%xmode plain

import os
import sys
import numpy as np

from texter.utils import text_utils as tu
from texter.models.keras.vanilla_cnn import vanilla_cnn
from texter.utils import embedding_utils as eu

from nltk.corpus import stopwords

from keras.layers import Embedding
from keras.callbacks import EarlyStopping

np.random.seed(15101993)

Exception reporting mode: Plain


In [3]:
excludes = stopwords.words("english")
text_path = "../../../citi/citi_rest/data/raw_data/"
text_labels = os.listdir(text_path)
pretrained_embeddings_path = "../../data/others/glove.6B.50d.txt"

In [4]:
df = tu.load_data(excludes, text_path, text_labels)
texts = [t for t in df.text]
labels = [l for l in df.label]

In [5]:
data, lbls, tokenizer, sequences, word_index = tu.keras_tokenizer(1000, texts, labels, 2000)

Found 66 unique tokens.
Shape of data tensor: (11, 2000)
Shape of label tensor: (11, 3)


In [15]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(data, lbls)

In [7]:
emb_lay = eu.glove_embedding_layer(pretrained_embeddings_path, word_index, EMBEDDING_DIM=50)

Found 400000 word embeddings


In [8]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, \
                          verbose=1, mode='auto')
callbacks_list = [earlystop]

In [18]:
model = vanilla_cnn(2000, 3, emb_lay)

In [19]:
model.fit(x_train, y_train, batch_size=256,
          epochs=1, validation_data=(x_val, y_val), callbacks=callbacks_list)

Train on 8 samples, validate on 3 samples
Epoch 1/1
8/8 [==============================] - 0s 55ms/step - loss: 1.0996 - acc: 0.2500 - val_loss: 1.0983 - val_acc: 0.3333
